In [267]:
import pandas as pd

In [268]:
# filter for:
# - files with a rating above X
# - files with no secondary labels (VERY BIG MUST)

In [269]:
data_2020 = pd.read_csv("../datasets/birdsongs-combined/metadata/train_metadata_2020.csv")
data_2020 = data_2020[["ebird_code", "filename", "rating", "secondary_labels"]]
data_2020["primary_label"] = data_2020["ebird_code"]
data_2020 = data_2020.drop("ebird_code", axis=1)

In [270]:
data_2021 = pd.read_csv("../datasets/birdsongs-combined/metadata/train_metadata_2021.csv")
data_2021 = data_2021[["primary_label", "filename", "rating", "secondary_labels"]]

In [271]:
data_2022 = pd.read_csv("../datasets/birdsongs-combined/metadata/train_metadata_2022.csv")
data_2022 = data_2022[["primary_label", "filename", "rating", "secondary_labels"]]

In [272]:
data_2023 = pd.read_csv("../datasets/birdsongs-combined/metadata/train_metadata_2023.csv")
data_2023 = data_2023[["primary_label", "filename", "rating", "secondary_labels"]]

In [273]:
data_2024 = pd.read_csv("../datasets/birdsongs-combined/metadata/train_metadata_2024.csv")
data_2024 = data_2024[["primary_label", "filename", "rating", "secondary_labels"]]

In [274]:
total_data = pd.concat([
    data_2020,
    data_2021,
    data_2022,
    data_2023,
    data_2024,
], axis=0).reset_index(drop=True)

print(len(total_data))
print(len(total_data[total_data["secondary_labels"] == "[]"]))

total_data

140501
105085


,filename,rating,secondary_labels,primary_label
0,XC134874.mp3,3.5,"['Empidonax minimus_Least Flycatcher', 'Leioth...",aldfly
1,XC135454.mp3,4.0,[],aldfly
2,XC135455.mp3,4.0,[],aldfly
3,XC135456.mp3,3.5,"['Dumetella carolinensis_Gray Catbird', 'Bomby...",aldfly
4,XC135457.mp3,4.0,[],aldfly
...,...,...,...,...
140496,zitcis1/XC845747.ogg,5.0,[],zitcis1
140497,zitcis1/XC845817.ogg,4.0,[],zitcis1
140498,zitcis1/XC856176.ogg,4.0,[],zitcis1
140499,zitcis1/XC856723.ogg,4.5,[],zitcis1


In [275]:
total_data["xc_id"] = [i.split("/")[-1][:-4][2:] for i in total_data["filename"]]
total_data["filetype"] = [i.split("/")[-1][-3:] for i in total_data["filename"]]
total_data = total_data.drop("filename", axis=1)

In [276]:
total_data

,rating,secondary_labels,primary_label,xc_id,filetype
0,3.5,"['Empidonax minimus_Least Flycatcher', 'Leioth...",aldfly,134874,mp3
1,4.0,[],aldfly,135454,mp3
2,4.0,[],aldfly,135455,mp3
3,3.5,"['Dumetella carolinensis_Gray Catbird', 'Bomby...",aldfly,135456,mp3
4,4.0,[],aldfly,135457,mp3
...,...,...,...,...,...
140496,5.0,[],zitcis1,845747,ogg
140497,4.0,[],zitcis1,845817,ogg
140498,4.0,[],zitcis1,856176,ogg
140499,4.5,[],zitcis1,856723,ogg


In [277]:
# remove files with any secondary labels
total_data = total_data[total_data["secondary_labels"] == "[]"].drop("secondary_labels", axis=1).reset_index(drop=True)

total_data

,rating,primary_label,xc_id,filetype
0,4.0,aldfly,135454,mp3
1,4.0,aldfly,135455,mp3
2,4.0,aldfly,135457,mp3
3,4.0,aldfly,135459,mp3
4,4.0,aldfly,135460,mp3
...,...,...,...,...
105080,5.0,zitcis1,845747,ogg
105081,4.0,zitcis1,845817,ogg
105082,4.0,zitcis1,856176,ogg
105083,4.5,zitcis1,856723,ogg


In [278]:
total_data = total_data.drop_duplicates().reset_index(drop=True)

In [280]:
indices_to_drop = []

for index in sorted([i for i, v in total_data["xc_id"].duplicated().items() if v]):

    duplicate_frame = total_data[total_data["xc_id"] == total_data.iloc[index, :]["xc_id"]]

    # if classification changed, remove all that aren't the most recent classification
    for i_to_drop in duplicate_frame[duplicate_frame["primary_label"] != duplicate_frame.iloc[-1, :]["primary_label"]].index:
        indices_to_drop.append(i_to_drop)

    duplicate_frame = duplicate_frame[duplicate_frame["primary_label"] == duplicate_frame.iloc[-1, :]["primary_label"]]
    if len(duplicate_frame) > 1:
        for i_to_drop in duplicate_frame.index[:-1]:
            indices_to_drop.append(i_to_drop)

total_data = total_data.drop(indices_to_drop)

KeyboardInterrupt: 

In [283]:
len(sorted([i for i, v in total_data["xc_id"].duplicated().items() if v]))

9618